In [1]:
import modules.model_mobilenetv3 as mobilenetv3_original
import modules.model_mobilenetv3_mini as mobilenetv3_mini

import numpy as np
import torch
from torchinfo import summary

import matplotlib.pyplot as plt

# Profiler: torchprofile

In [2]:
from torchprofile import profile_macs

# Profiler: torchlop

In [3]:
from torchlop import profile

In [4]:
def get_macs_and_params_per_layer(model):

    dummy_inp = torch.randn(1, 3, 224, 224)
    macs, params, layer_infos = profile(model, inputs=(dummy_inp, ))

    print('Total MACs of {}: {:.4g} M'.format(model.__class__.__name__, macs / 1e6))
    print('Total params of {}: {}'.format(model.__class__.__name__, params))

    macs_layer = {}
    params_layer = {}

    conv_idx = 0
    linear_idx = 0
    
    for i, k in enumerate(layer_infos.keys()):
        layer_type = layer_infos[k]["type"]
        layer_ops = layer_infos[k]["ops"]
        layer_params = layer_infos[k]["params"]
        if layer_ops != 0 and layer_type != "BatchNorm2d":
            if layer_type == "Conv2d": 
                macs_layer.update({layer_type + '_' + str(conv_idx): layer_ops})   
                params_layer.update({layer_type + '_' + str(conv_idx): layer_params}) 
                conv_idx += 1
            elif layer_type == "Linear":
                macs_layer.update({layer_type + '_' + str(linear_idx): layer_ops}) 
                params_layer.update({layer_type + '_' + str(linear_idx): layer_params})  
                linear_idx += 1 
            else:
                macs_layer.update({layer_type: layer_ops})
                params_layer.update({layer_type: layer_params})

    return macs_layer, params_layer

## Plot results

In [5]:
def plot_results(res_layer, res_type: str):
    fig, ax = plt.subplots(figsize=(10,3))

    layers_name = list(res_layer.keys())
    layer_res = list(res_layer.values())
    
    rects = ax.bar(layers_name, layer_res)
    ax.bar_label(rects, padding=3, fontsize=7)
    if res_type == "macs":
        ylabel = "MACs (M)"
        title = "MACs per layer (M)"
    elif res_type == "params":
        ylabel = "#Params"  
        title = "#Params per layer"
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.set_xticks(np.arange(len(layers_name)), layers_name, rotation=90, fontsize=8)
    
    plt.show()

# Models

In [6]:
model_original = mobilenetv3_original.MobileNetV3_Small(num_classes=2).to('cpu')
model_mini = mobilenetv3_mini.MobileNetV3_Small(num_classes=2).to('cpu')

In [7]:
model_original.eval();
model_mini.eval();

## MobileNetV3 Original

### Profile

In [8]:
# inputs = torch.randn(1, 3, 224, 224)

# macs = profile_macs(model_original, inputs)
# print('{}: {:.4g} M'.format(model_original.__class__.__name__, macs / 1e6))

In [9]:
# macs_mbnet, params_mbnet = get_macs_and_params_per_layer(model_original)

In [10]:
# plot_results(macs_mbnet, res_type="macs")

In [11]:
# plot_results(params_mbnet, res_type="params")

### Summary

In [12]:
print(summary(model_original, input_size=(1, 3, 224, 224)))

Layer (type:depth-idx)                             Output Shape              Param #
MobileNetV3_Small                                  [1, 2]                    --
├─Conv2d: 1-1                                      [1, 16, 112, 112]         432
├─BatchNorm2d: 1-2                                 [1, 16, 112, 112]         32
├─Hardswish: 1-3                                   [1, 16, 112, 112]         --
├─Sequential: 1-4                                  [1, 96, 7, 7]             --
│    └─Block: 2-1                                  [1, 16, 56, 56]           --
│    │    └─Conv2d: 3-1                            [1, 16, 112, 112]         256
│    │    └─BatchNorm2d: 3-2                       [1, 16, 112, 112]         32
│    │    └─ReLU: 3-3                              [1, 16, 112, 112]         --
│    │    └─Conv2d: 3-4                            [1, 16, 56, 56]           144
│    │    └─BatchNorm2d: 3-5                       [1, 16, 56, 56]           32
│    │    └─ReLU: 3-6           

In [13]:
model_original.to('cpu')
dummy_in = torch.randn(4, 3, 224, 224)
out = model_original(dummy_in)

In [14]:
out

tensor([[ 3.4131e-10, -5.1045e-10],
        [ 4.8862e-10, -4.9142e-10],
        [ 4.7172e-10, -4.5702e-10],
        [ 4.3834e-10, -4.4543e-10]], grad_fn=<AddmmBackward0>)

## MobilenetV3 Mini

### Summary

In [15]:
print(summary(model_mini, input_size=(1, 3, 224, 224)))

Layer (type:depth-idx)                             Output Shape              Param #
MobileNetV3_Small                                  [1, 2]                    --
├─Conv2d: 1-1                                      [1, 16, 112, 112]         432
├─BatchNorm2d: 1-2                                 [1, 16, 112, 112]         32
├─Hardswish: 1-3                                   [1, 16, 112, 112]         --
├─Sequential: 1-4                                  [1, 48, 14, 14]           --
│    └─Block: 2-1                                  [1, 16, 56, 56]           --
│    │    └─Conv2d: 3-1                            [1, 16, 112, 112]         256
│    │    └─BatchNorm2d: 3-2                       [1, 16, 112, 112]         32
│    │    └─ReLU: 3-3                              [1, 16, 112, 112]         --
│    │    └─Conv2d: 3-4                            [1, 16, 56, 56]           144
│    │    └─BatchNorm2d: 3-5                       [1, 16, 56, 56]           32
│    │    └─ReLU: 3-6           

### ONNX export

In [19]:
model_mini.cpu();

In [20]:
torch_input = torch.randn(1, 3, 224, 224)

In [21]:
torch.onnx.export(
    model_mini,                   # The model to be exported
    torch_input,            # The sample input tensor
    "MobilenetV3_MINI.onnx",            # The output file name
    export_params=True,      # Store the trained parameter weights inside the model file
    do_constant_folding=False,  # Whether to execute constant folding for optimization
    input_names=['input'],     # The model's input names
    output_names=['output'],   # The model's output names
)